## Pytorch Tutorials : Build the Neural Network
- [Pytorch Tutorials](https://pytorch.org/tutorials/)
- [Build the Neural Network](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

### Build the Neural Network
신경망은 데이터에 대한 연산을 수행하는 계층/모듈로 구성  

`torch.nn` 네임스페이스는 자신만의 신경망을 구축하는 데 필요한 모든 구성 요소를 제공합니다. 

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training

In [2]:
# gpu 가용성 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


### Define the Class
`nn.Module`을 하위 분류하여 신경망 모델을 정의한다. 또한, `__init__`의 신경망 레이어를 초기화 한다.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

신경망 인스턴스를 만들어서 장치로 옮기고 그 구조를 확인

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터 전달 > 모델의 순전파 연산과 일부 역전파 연산이 실행됨 (굳이 model.forward() 호출하지 않아도 됨)  

모델을 호출하면 각 클래스에 대해 예측 값을 갖는 10차원 텐서가 반환된다. 우리는 nn.Softmax module을 통과시킴으로써 예측 확률을 얻는다

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


### Model Layers
FashionMNIST 모델의 레이어를 분해해 봅시다. 이를 설명하기 위해 28x28 사이즈의 이미지 3개로 구성된 미니배치 샘플을 채취하여 네트워크를 통과할 때 어떤 일이 일어나는지 알아보겠습니다.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten()  
레이어를 초기화하여 각 2D 28x28 이미지를 784픽셀 값의 연속 배열로 변환 (미니 배치 차원(dim=0)이 유지됨).

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear()  
그 저장 가중치와 바이어스를 이용하여 상기 입력에 선형 변환을 적용시키는 모듈

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU()  
비선형 활성화는 모델의 입력과 출력 사이에 복잡한 매핑을 생성합니다. 선형 변환 후에 적용하여 비선형성을 도입 하여 신경망이 다양한 현상을 학습하도록 돕습니다.

이 모델에서는 선형 레이어 사이에 nn.ReLU 를 사용 하지만 모델에 비선형성을 도입하기 위한 다른 활성화가 있습니다.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2244, -0.0535, -0.5564,  0.0033, -0.2413,  0.3353,  0.9000, -0.6146,
          0.1706, -0.3343, -0.1026,  0.6228,  0.0440,  0.3048, -0.2537,  0.2835,
         -0.0936,  0.2472, -0.2011, -0.1276],
        [-0.2678,  0.0575, -0.3276,  0.1067, -0.5123,  0.5603,  0.6656, -0.3501,
          0.1722, -0.3640, -0.0359,  0.6597,  0.1392, -0.0702, -0.3290,  0.5670,
         -0.0970,  0.2720,  0.0648, -0.1461],
        [ 0.0612, -0.1240, -0.3355,  0.0852, -0.4117,  0.2753,  0.8200, -0.3926,
          0.0762, -0.4082, -0.1469,  0.7591,  0.1468,  0.2095, -0.1065,  0.3736,
          0.4928,  0.2663, -0.0858,  0.0950]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0033, 0.0000, 0.3353, 0.9000, 0.0000, 0.1706,
         0.0000, 0.0000, 0.6228, 0.0440, 0.3048, 0.0000, 0.2835, 0.0000, 0.2472,
         0.0000, 0.0000],
        [0.0000, 0.0575, 0.0000, 0.1067, 0.0000, 0.5603, 0.6656, 0.0000, 0.1722,
         0.0000, 0.0000, 0.6597, 0.1392, 0.0000, 0.000

nn.Sequential()  
모듈의 순서가 지정된 컨테이너로, 데이터는 정의된 것과 동일한 순서로 모든 모듈을 통해 전달됩니다.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax()  
신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달되는 [-infty, infty]의 원시 값인 로짓을 반환 합니다. 로짓은 각 클래스에 대한 모델의 예측 확률을 나타내는 값 [0, 1]로 조정됩니다. 매개변수 `dim`는 값의 합이 1이 되어야 하는 차원을 나타냅니다.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### 모델 매개변수
신경망 내부의 많은 레이어는 매개변수화되어 있습니다 . 즉, 훈련 중에 최적화된 관련 가중치와 편향이 있습니다. nn.Module은 모델 개체 내부에 정의된 모든 필드를 자동으로 추적하고 모델 parameters()또는 named_parameters()메서드를 사용하여 모든 매개변수에 액세스할 수 있도록 합니다.

In [12]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0280, -0.0300,  0.0334,  ..., -0.0253,  0.0347, -0.0325],
        [ 0.0040, -0.0262,  0.0309,  ..., -0.0012, -0.0200, -0.0329]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0103, -0.0052], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0082,  0.0017,  0.0385,  ...,  0.0134,  0.0437,  0.0043],
        [-0.0110,  0.0001,  0.0363,  ..., -0.0052,  0.0169, -0.0392]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | S